# Data Processing

In [14]:
import pandas as pd
import numpy as np
# model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error

In [13]:
# Load data
def load_data(dataset = 'df_19_24_cleaned'):
    data = pd.read_pickle(f'../data/{dataset}.pkl') 
    print(data.info())
    return data

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
def data_scaler(data):
    data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
    print('Data is scaled')
    return data_scaled

In [12]:
# 1. Train-Test Split (keeping all hourly data points in the last 7 days of each month for testing)
def train_test_split_7(data):
    test_indices = data.index.to_series().groupby([data.index.year, data.index.month]).apply(lambda x: x[-24*7:])
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    print(f'Shape of train_data: {train_data.shape}')
    print(f'Shape of test_data: {test_data.shape}')
    return train_data, test_data

## LSTM

In [27]:
def create_sequences(data, seq_length=24, target_column='price'):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Include all features in X
        y.append(data[target_column].iloc[i+seq_length])  # Target is still the original 'price'
    return np.array(X), np.array(y)

def create_sequences_2(data, seq_length=24, features = ['price']):
    # features = ['price', 'wind_energy_generation', 'solar_energy_generation', 'total_load']
    
    # Convert to numpy array for easier slicing
    data_array = data[features].values
    
    # Initialize lists for sequences and labels
    sequences = []
    labels = []
    
    # Create sequences
    for i in range(len(data_array) - seq_length):
        # Sequence of 24 time steps
        sequences.append(data_array[i:i + seq_length])
        
        # The label is the price at the next time step after the sequence
        labels.append(data_array[i + seq_length, 0])  # Assuming `price` is the first column
    
    # Convert lists to numpy arrays
    sequences = np.array(sequences)
    labels = np.array(labels)

    return  sequences, labels

In [15]:
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, activation='relu', input_shape=input_shape),
        Dropout(0.2),  # Dropout to prevent overfitting
        Dense(1)  # Output layer with a single neuron for regression
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

## Evaluation

In [17]:
# Define sMAPE function for evaluation
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [28]:
def scaler_inverse(y_test_scaled, y_preds_scaled, X_test):
    y_test_original = scaler.inverse_transform(
        np.concatenate((y_test_scaled.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]

    y_preds_original = scaler.inverse_transform(
        np.concatenate((y_preds_scaled, X_test[:, -1, 1:]), axis=1))[:, 0]

    return y_test_original, y_preds_original
    
def scaler_inverse_2(y_test_scaled, y_preds_scaled, num_features = 1):
    # Reshape predictions and true values for inverse transformation
    y_preds_scaled = y_preds_scaled.reshape(-1, 1)
    y_test_scaled = y_test_scaled.reshape(-1, 1)
    
    # Extend with zeros for other features to match scaler's input shape
    # num_features = len(features)
    zeros = np.zeros((len(y_preds_scaled), num_features - 1))
    predictions_extended = np.concatenate([y_preds_scaled, zeros], axis=1)
    # test
    y_test_extended = np.concatenate([y_test, zeros], axis=1)
    
    # Inverse transform
    y_preds_original = scaler.inverse_transform(predictions_extended)[:, 0]  # Only take price column
    y_test_original = scaler.inverse_transform(y_test_extended)[:, 0]      

    return y_test_original, y_preds_original


In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def eva(y_test, y_pred, X_test):
    
    # Inverse scale predictions and actual values
    y_pred_rescaled = scaler.inverse_transform(
        np.concatenate((y_pred, X_test[:, -1, 1:]), axis=1)
    )[:, 0]
    y_test_rescaled = scaler.inverse_transform(
        np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1)
    )[:, 0]
    
    # Calculate MAE
    mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
                   
    def smape(y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    
    smape_value = smape(y_test_rescaled, y_pred_rescaled)
    print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value:.2f}")
    return y_test_rescaled, y_pred_rescaled

In [12]:
def eva_s(y_test_rescaled, y_pred_rescaled):
    
    # Calculate MAE
    mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
                   
    def smape(y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    
    smape_value = smape(y_test_rescaled, y_pred_rescaled)
    print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value:.2f}")

    return mae, rmse, smape_value
    # return y_test_rescaled, y_pred_rescaled

# EWT

In [23]:
import ewtpy

# data: https://github.com/vrcarva/ewtpy/blob/master/ewtpy/ewtpy.py

def ewt_decompose(data, K,  N = 5, log = 0, detect = "locmax", completion = 0, reg = 'average', lengthFilter = 10,sigmaFilter = 5):
    ewt,  mfb ,boundaries = ewtpy.EWT1D(data, 
                                        N = K, 
                                        log = log, 
                                        detect = detect, 
                                        completion = completion, 
                                        reg = reg, 
                                        lengthFilter = lengthFilter,
                                        sigmaFilter = sigmaFilter)
    

    combined_signal = np.sum(ewt, axis=1)
    return combined_signal, ewt


def plot_ewt(ewt, label = None ,start =None, end = None, ): 
    n = ewt.shape[1]
    fig, axes = plt.subplots(n, 1, figsize=(12, 9))
    for i in range(n):
        axes[i].plot(ewt[start:end,i])
        # axes[i].set_title(f'{name} EWT Component {i + 1}')
    
    # Set a shared ylabel for the entire plot
    fig.text(-0.001, 0.5, label, va='center', rotation='vertical', fontsize=12)
    
    plt.tight_layout()
    plt.show()


import numpy as np
import ewtpy
import matplotlib.pyplot as plt
from pywt import threshold

# T = 1000
# t = np.arange(1, T+1) / T
# f = np.cos(2 * np.pi * 0.8 * t) + 2 * np.cos(2 * np.pi * 10 * t) + 0.8 * np.cos(2 * np.pi * 100 * t)

def ewt_sureshrink_denoise(data, K, start=None, end=None, plot=False):
    # Perform EWT decomposition
    ewt, mfb, boundaries = ewtpy.EWT1D(data, N=K)
    
    # Apply SureShrink thresholding to each component
    denoised_components = np.zeros_like(ewt)
    for i in range(K):
        component = ewt[:, i]
        
        # Calculate threshold based on the component's median absolute deviation (MAD)
        sigma = np.median(np.abs(component - np.median(component))) / 0.6745
        threshold_val = sigma * np.sqrt(2 * np.log(len(component)))
        
        # Apply soft thresholding to the component
        denoised_components[:, i] = threshold(component, value=threshold_val, mode='soft')
    
    # Reconstruct the denoised signal by summing the denoised components
    denoised_signal = np.sum(denoised_components, axis=1)
    
    # Plotting each component (optional)
    if plot:
        fig, axes = plt.subplots(K + 1, 1, figsize=(12, 9))
        for i in range(K):
            axes[i].plot(denoised_components[start:end, i], label=f'Denoised EWT Component {i + 1}')
            axes[i].legend()
        axes[-1].plot(denoised_signal[start:end], label='Reconstructed Signal', color='black')
        axes[-1].legend()
        plt.tight_layout()
        plt.show()
    
    return denoised_signal, denoised_components

In [ ]:
# def ewt_decompose(signal, K):
#     # Perform Fourier Transform
#     fourier_transform = fft(signal)
#     components = []
#     freq_range = np.linspace(0, np.pi, K + 1)
    
#     # Define filters and extract components
#     for i in range(K):
#         filter_mask = np.zeros_like(fourier_transform)
#         left_boundary = freq_range[i]
#         right_boundary = freq_range[i + 1]
        
#         for j in range(len(fourier_transform)):
#             frequency = j * np.pi / len(fourier_transform)
#             if left_boundary <= frequency <= right_boundary:
#                 filter_mask[j] = 1
        
#         # Apply filter and inverse FFT
#         component_fft = fourier_transform * filter_mask
#         component_ifft = ifft(component_fft).real  # Convert back to time domain
#         components.append(component_ifft)
    
#     return np.array(components).T  # Shape: (t

## Stat Tests

In [1]:
from statsmodels.tsa.stattools import adfuller

In [2]:
def adf_test(data):
    adf_test = adfuller(data, regression='c')
    print('ADF Statistic: {:.6f}\np-value: {:.6f}\n#Lags used: {}'
          .format(adf_test[0], adf_test[1], adf_test[2]))
    for key, value in adf_test[4].items():
        print('Critical Value ({}): {:.6f}'.format(key, value))

## Plot

In [7]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
def plot_acf_pacf(data):
    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10, 6))
    plot_acf(data, lags=50, ax=ax1)
    plot_pacf(data, lags=50, ax=ax2)
    plt.tight_layout()
    plt.show()

In [9]:
def compare_preds(test, preds):
    fig, ax = plt.subplots(figsize = (12,6))
    # ax.plot(train['date'], train['data'], 'g-.', label='Train')
    ax.plot(test, 'b-', label='Test')
    ax.plot(preds, 'r--', label='Predicted')
    ax.set_xlabel('Date')
    ax.set_ylabel('Electricity Price')
    # ax.axvspan(80, 83, color='#808080', alpha=0.2)
    ax.legend(loc=2)
    
    
    fig.autofmt_xdate()
    plt.tight_layout()
